In [ ]:
try:
  # Use the %tensorflow_version magic if in colab.
  %tensorflow_version 2.x
except Exception:
  pass

import tensorflow as tf
import time
import numpy as np
import os
import matplotlib.pyplot as plt
from datetime import datetime
print('done')


In [ ]:
AUTOTUNE = tf.data.experimental.AUTOTUNE
BATCH_SIZE=64
IMG_HEIGHT=96
start = time.time()
CLASS_NAMES=[str(i) for i in range(1,103)]
#image_generator = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)
data_dir = "/kaggle/input/pytorch-challange-flower-dataset/flower_data/flower_data/train"
list_ds = tf.data.Dataset.list_files(str(data_dir+'/*/*'))
def get_label(file_path):
  # convert the path to a list of path components
  parts = tf.strings.split(file_path, os.path.sep)
  # The second to last is the class-directory
  return int(parts[-2])
def decode_img(img):
  # convert the compressed string to a 3D uint8 tensor
  img = tf.image.decode_jpeg(img, channels=3)
  # Use `convert_image_dtype` to convert to floats in the [0,1] range.
  img = tf.image.convert_image_dtype(img, tf.float32)
  # resize the image to the desired size.
  return tf.image.resize(img, [IMG_HEIGHT, IMG_HEIGHT])
def process_path(file_path):
  label = get_label(file_path)
  # load the raw data from the file as a string
  img = tf.io.read_file(file_path)
  img = decode_img(img)
  return img, label
# Set `num_parallel_calls` so multiple images are loaded/processed in parallel.
labeled_ds = list_ds.map(process_path, num_parallel_calls=AUTOTUNE)

def prepare_for_training(ds, cache=True, shuffle_buffer_size=1000):
  # This is a small dataset, only load it once, and keep it in memory.
  # use `.cache(filename)` to cache preprocessing work for datasets that don't
  # fit in memory.
  if cache:
    if isinstance(cache, str):
      ds = ds.cache(cache)
    else:
      ds = ds.cache()

  ds = ds.shuffle(buffer_size=shuffle_buffer_size)

  # Repeat forever
  ds = ds.repeat()

  ds = ds.batch(BATCH_SIZE)

  # `prefetch` lets the dataset fetch batches in the background while the model
  # is training.
  ds = ds.prefetch(buffer_size=AUTOTUNE)

  return ds

train_ds = prepare_for_training(labeled_ds)
end=time.time()
print((end-start)*1000/60)
test_dir = "/kaggle/input/pytorch-challange-flower-dataset/flower_data/flower_data/valid"
list_ds = tf.data.Dataset.list_files(str(test_dir+'/*/*'))
labeled_ds = list_ds.map(process_path, num_parallel_calls=AUTOTUNE)
test_ds = prepare_for_training(labeled_ds)

In [ ]:
print(len(labeled_ds))

In [ ]:
for image, label in labeled_ds.take(1):
  print("Image shape: ", type(image))
  print("Label: ", label.numpy())
print(CLASS_NAMES)

In [ ]:
for image_batch, label_batch in train_ds.take(1):
   pass

image_batch.shape

In [ ]:
"""
model = tf.keras.Sequential(layers)
model.compile(optimizer=tf.optimizers.Adam(),
             loss=tf.losses.SparseCategoricalCrossentropy(),
              metrics=[tf.metrics.SparseCategoricalAccuracy()])
"""
base_model = tf.keras.applications.MobileNetV2(input_shape=(96, 96, 3),
                                              include_top=False,
                                               weights='imagenet')


In [ ]:
feature_batch = base_model(image_batch)
print(feature_batch.shape)

In [ ]:
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
feature_batch_average = global_average_layer(feature_batch)
print(feature_batch_average.shape)

In [ ]:
prediction_layer = tf.keras.layers.Dense(103)
prediction_batch = prediction_layer(feature_batch_average)
print(prediction_batch.shape)

In [ ]:
model = tf.keras.Sequential([
  base_model,
  global_average_layer,
  prediction_layer
])

In [ ]:
base_learning_rate = 0.0001
model.compile(optimizer=tf.keras.optimizers.Adam(),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])

In [ ]:
logdir="logs/" + datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=logdir)

In [ ]:
history=model.fit(train_ds,steps_per_epoch=50,epochs=1,validation_data=test_ds,validation_steps=50,callbacks=[tensorboard_callback])

In [ ]:
%reload_ext tensorboard
%tensorboard --logdir logs

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

plt.figure(figsize=(8, 8))
plt.subplot(2, 1, 1)
plt.plot(acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.ylabel('Accuracy')
plt.ylim([min(plt.ylim()),1])
plt.title('Training and Validation Accuracy')

plt.subplot(2, 1, 2)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.ylabel('Cross Entropy')
plt.ylim([4.0,6.0])
plt.title('Training and Validation Loss')
plt.xlabel('epoch')
plt.show()

In [ ]:
import cv2
train_path = "/kaggle/input/pytorch-challange-flower-dataset/flower_data/flower_data/valid/"
image_files = os.listdir(train_path)
train_images=[]
train_labels=[]
def load_image(file_path):
    return cv2.imread(file_path)
for folder in image_files:
  for file in os.listdir(train_path+"/"+folder):
    train_images.append(load_image(train_path +folder+ "/"+file))
    #train_images.append(file)
    train_labels.append(int(folder))
print('done')
def preprocess_image(img, side=96):
    min_side = min(img.shape[0], img.shape[1])
    img = img[:min_side, :min_side]
    img = cv2.resize(img, (side,side))
    return img / 255.0
for i in range(len(train_images)):
    train_images[i] = preprocess_image(train_images[i])
train_images=np.array(train_images)
train_labels = np.array(train_labels)
eval_predictions=model.predict(train_images)
result=[]
crct =0 
for i in range(len(train_labels)):
    result.append(np.argmax(eval_predictions[i]))
    if result[i]==train_labels[i]:
        print(result[i],train_labels[i],"yes")
        crct=crct+1 
    else:
        print(result[i],train_labels[i],"no")
print(crct/len(eval_predictions))


In [ ]:
print(crct)